# Distance calculate


In [133]:
import numpy as np
import cv2
from PIL import Image 
from ultralytics import YOLO
import cv2
import math

In [134]:
filepath = r"D:\python\Distance_measurement\CheckerBoards\3.jpg"

img = Image.open (filepath)

width, height = img.size

print ("The dimensions of the image are:", width, "x", height)

The dimensions of the image are: 2592 x 1944


##### Define camera parameters

In [135]:
fx = 2440.9540138328043
fy = 2434.6728574963054
cx = 1944
cy = 2592
ppx = 1248.1362594555521
ppy = 957.0167856547552
mtx = np.array([[fx, 0, cx],
                [0, fy, cy],
                [0, 0, 1]])
# focal length
f = (fx + fy) / 2
display(mtx)

array([[       2441,           0,        1944],
       [          0,      2434.7,        2592],
       [          0,           0,           1]])

##### Define distortion parameters

In [136]:
rk1 = 0.10461012298345916
rk2 = -0.005435444683453119
rk3 = -1.0492719403629378
tp1 = 0.0006065919889218954
tp2 = 0.004444262737182748

##### Getting xmax, ymax, xmin, ymin using YOLOv8

In [137]:
model = YOLO("yolov8n.pt")


In [138]:
img = cv2.imread(r'D:\python\Distance_measurement\raw_images\medium\medium0.jpg')
# cv2.imshow("image", img)
# cv2.waitKey(0)

##### Applying YOLOv8

In [139]:
CONFIDENCE_THRESHOLD = 0.8
GREEN = (0, 255, 0)

detections = model(img, verbose=False)[0]
for data in detections.boxes.data.tolist():
        # extract the confidence (i.e., probability) associated with the detection
        confidence = data[4]

        # filter out weak detections by ensuring the 
        # confidence is greater than the minimum confidence
        if float(confidence) < CONFIDENCE_THRESHOLD:
            continue

        # if the confidence is greater than the minimum confidence,
        # draw the bounding box on the frame
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        cv2.rectangle(img, (xmin, ymin) , (xmax, ymax), GREEN, 2)
cv2.imshow("Frame", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(xmin, xmax, ymin, ymax)

156 291 67 264


##### Object depth estimation

In [140]:
xf = 0.5 * (xmin + xmax)
yf = ymax
display(xf, yf)

223.5

264

In [141]:
xn = (xf - cx) / fx
yn = (yf - cy) / fy
display(xn, yn)

-0.7048473630596825

-0.9561859585496828

P: primary point in 2D image<br>
F: Human foot point in 2D image<br>
F: corresponding point of P in 3D<br>
F': corresponding Human foot point in 3D<br>

In [142]:
H       = 1                             # Camera height
alpha   = math.radians(18)              # Camera angle
Zp      = H * (1 / math.cos(alpha))     # Zp = OP'
display(Zp)

1.0514622242382672

In [143]:
P = (cx / 2, cy / 2)
B = (0, P[1])
A = (P[0], 0)
O = (P[0], P[1], H)
BP = math.sqrt((P[0] - B[0])**2 + (P[1] - B[1])**2)
AP = math.sqrt((P[0] - A[0])**2 + (P[1] - A[1])**2)
OP = f
F = (xf, yf)
tan_BOP = BP / OP
tan_AOP = AP / OP
display(BP, AP, OP)


972.0

1296.0

2437.813435664555

In [144]:
dx = 1000 * f * tan_BOP
dy = 1000 * f * tan_AOP
sx = cx / dx
sy = cy / dy

d = math.sqrt(((cx - xf)/sx)**2 + ((cy -yf)/sy)**2)
teta = math.atan(d / f)
Z = Zp * (1/math.cos(teta))
xc = xn * Z
yc = yn * Z
zc = Z  

In [145]:
# Angle of heading with respect to North: p
p = math.radians(10)

xb = xc
yb = yc*math.cos(alpha)
zb = -yc*math.sin(alpha) + zc*math.cos(alpha)

E = zb*math.sin(p) + xb*math.cos(p)
N = zb*math.cos(p) - xb*math.sin(p)
U = -yb

display(E, N, U)

-298.2056824702954

842.7706043507735

567.7104146997771

In [146]:
b = math.atan(abs(E / N))
s = math.sqrt(E**2 + N**2)
dX = s*math.sin(b)
dY = s*math.cos(b)

In [147]:
UAV_lat = 21.0042302
UAB_lon = 105.8426075

delta_lo = dX / (11320 * math.cos(math.radians(UAB_lon)))
delta_la = dY / 110540

In [148]:
laf = UAV_lat + delta_la
lof = UAB_lon + delta_lo
display(laf, lof)

21.011854323433603

105.74611053478255